# *MAIN INPUT INFORMATION FOR FORMATE OUTPUT RASTERS*

In [ ]:
#  RN contamination of trunk wood
PL_Sr = int(input('Enter the permissible level for Sr-90 activity concentration in trunk wood (Bq/kg):'))
PL_Cs = int(input('Enter the permissible level for Cs-137 activity concentration in trunk wood (Bq/kg):'))


# Enter the year for which you want to get indicators (from 2020 to 2050)
year = int(input('Enter the year for which you want to receive indicators (from 2020 to 2050):'))
if year>2019 and year<2051 :
    print('\n************** \n Ok! \n**************\n')
else:
    print('\n************** \n Error! \n**************\n')

# *Prepare all input data*
# Load modules and raster data

In [ ]:
# Import Python print function and division
from __future__ import print_function, division

# Import klepto, GDAL, NumPy, and matplotlib
import klepto
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Create local storage using 'klepto'
db = klepto.archives.dir_archive('db', serialized=True, cached=False)

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

In [ ]:
# Read our raster`s bands/channels
"""
Raster bands includes:
0: Sr-90 density contamination of soil, 
1: Cs-137 density contamination of soil, 
2: ambient equivalent dose rate (ADER),
3: species ID, 
4: origin ID, 
5: stand age, 
6: stand height, 
7: stand DBH, 
8: relative stock,
9: soil fertility ID, 
11: the humidity of the soil 
"""
img_ds = gdal.Open('./input_raster/clip_comb_bands.tif', gdal.GA_ReadOnly)

# Copy the transformation to a variable
GT_input = img_ds.GetGeoTransform()

img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
for b in range(img.shape[2]):
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()

# Clear up RAM
img_ds = ''

In [ ]:
# Show bands
plt.subplot(121)
plt.imshow(img[:, :, 0], cmap=plt.cm.Greys_r)
plt.title('Sr-90_kBq/sq.m')

plt.subplot(122)
plt.imshow(img[:, :, 1], cmap=plt.cm.Greys_r)
plt.title('Cs-137_kBq/sq.m')

plt.show()

# Estimation of SI (Site index)

In [ ]:
# Select bands for est. SI

db['X_SI'] = img[:, :, [3, 4, 5, 6]]

print('Our X matrix is sized: {sz}'.format(sz=db['X_SI'].shape))

In [ ]:
#  Importing the main library for building model and its analysis

import xgboost as xgb
import pandas as pd

In [ ]:
# Load learned model
xgb_model_SI = xgb.XGBRegressor()

xgb_model_SI.load_model('./models/growing_model/XGB_model_SI.txt')

In [ ]:
# Show all parameters of XGBoost model

print(xgb_model_SI)

In [ ]:
# Take our full image, ignore the Fmask band, and reshape into long 2d array (nrow * ncol, nband) for classification
db['new_shape_X_SI'] = (db['X_SI'].shape[0] * db['X_SI'].shape[1], db['X_SI'].shape[2])

db['array_X_SI'] = db['X_SI'].reshape(db['new_shape_X_SI'])
print('Reshaped from {o} to {n}'.format(o=db['X_SI'].shape,
                                        n=db['array_X_SI'].shape))

In [ ]:
# Create predict values for each pixels for Site index
db['pred_SI'] = xgb_model_SI.predict(db['array_X_SI'], ntree_limit=xgb_model_SI.best_ntree_limit)

# Reshape our predict map
db['output_pred_SI'] = db['pred_SI'].reshape(db['X_SI'][:, :, 0].shape)

# Save prediction SI map

In [ ]:
# Load "rasterio" library
import rasterio

In [ ]:
# Save output raster
from affine import Affine
af_transf = Affine.from_gdal(*GT_input)

with rasterio.open(
    './output_rasters/01_SI_map.tif',
    'w',
    driver='GTiff',
    height=db['output_pred_SI'].shape[0],
    width=db['output_pred_SI'].shape[1],
    count=1,
    dtype=np.float32,
    transform=af_transf
) as dest_file:
    dest_file.write(db['output_pred_SI'], 1)
dest_file.close()

# Calculate forest parameters

In [ ]:
# Calculate stands probobly parameters when RS/P = 1

# Formate array data for species_ID and origin_ID:
db['X_Sp_Or'] = img[:, :, [3, 4]]    

db['new_shape_norm_X_Sp_Or'] = (db['X_Sp_Or'].shape[0] * db['X_Sp_Or'].shape[1], db['X_Sp_Or'].shape[2])
db['array_norm_X_Sp_Or'] = db['X_Sp_Or'].reshape(db['new_shape_norm_X_Sp_Or'])
    
# Formate array data for A:
db['X_A'] = img[:, :, [5]]    

db['new_shape_norm_X_A'] = (db['X_A'].shape[0] * db['X_A'].shape[1], db['X_A'].shape[2])
db['array_norm_X_A'] = db['X_A'].reshape(db['new_shape_norm_X_A'])

In [ ]:
# Unit array data for est. forest normal stand parameters 
db['X_norm_FP'] = np.c_[db['array_norm_X_Sp_Or'], db['pred_SI'], db['array_norm_X_A']]

In [ ]:
# Show of united colomns
print(db['X_norm_FP'])

In [ ]:
# Load learned XGBoost models for est parameters of normal forest stands
xgb_model_D = xgb.XGBRegressor()
xgb_model_D.load_model('./models/growing_model/XGB_model_ln_D_THR_data.txt')

xgb_model_H = xgb.XGBRegressor()
xgb_model_H.load_model('./models/growing_model/XGB_model_ln_H_THR_data.txt')

xgb_model_G = xgb.XGBRegressor()
xgb_model_G.load_model('./models/growing_model/XGB_model_ln_G_THR_data.txt')

xgb_model_GS = xgb.XGBRegressor()
xgb_model_GS.load_model('./models/growing_model/XGB_model_ln_GS_THR_data.txt')

In [ ]:
# Create predict values for parameters as normal forest stands
db['pred_ln_D1'] = xgb_model_D.predict(db['X_norm_FP'], ntree_limit=xgb_model_D.best_ntree_limit)

db['pred_ln_H1'] = xgb_model_H.predict(db['X_norm_FP'], ntree_limit=xgb_model_H.best_ntree_limit)

db['pred_ln_G1'] = xgb_model_G.predict(db['X_norm_FP'], ntree_limit=xgb_model_G.best_ntree_limit)

db['pred_ln_GS1'] = xgb_model_GS.predict(db['X_norm_FP'], ntree_limit=xgb_model_GS.best_ntree_limit)

db['pred_D1'] = np.exp(db['pred_ln_D1'])

db['pred_H1'] = np.exp(db['pred_ln_H1'])

db['pred_G1'] = np.exp(db['pred_ln_G1'])

db['pred_GS1'] = np.exp(db['pred_ln_GS1'])

In [1]:
# Ration between empirical and normal values of forest indicators
db['D'] = img[:, :, [7]]
db['new_shape_D'] = (db['D'].shape[0] * db['D'].shape[1], db['D'].shape[2])
db['array_D'] = db['D'].reshape(db['new_shape_D'])
db['ratio_D'] = np.divide(db['array_D'][:,0], db['pred_D1'])

db['H'] = img[:, :, [6]]
db['new_shape_H'] = (db['H'].shape[0] * db['H'].shape[1], db['H'].shape[2])
db['array_H'] = db['H'].reshape(db['new_shape_H'])
db['ratio_H'] = np.divide(db['array_H'][:,0], db['pred_H1'])

db['P'] = img[:, :, [8]] # relative stocking
db['new_shape_P'] = (db['P'].shape[0] * db['P'].shape[1], db['P'].shape[2])
db['array_P'] = db['P'].reshape(db['new_shape_P'])

db['G'] = np.multiply(db['array_P'][:,0], db['pred_G1'])
db['GS'] = np.multiply(db['array_P'][:,0], db['pred_GS1'])

NameError: name 'img' is not defined

In [ ]:
# Calculate forest  parameters for years
a_stand = year-2016

A_forecast = array_norm_X_A[:,0]+a_stand # stand age
FP_forecast = np.c_[array_norm_X_Sp_Or, pred_SI, A_forecast]

# Create predict values for parameters as normal forest stands
pred_ln_D2 = xgb_model_D.predict(FP_forecast, ntree_limit=xgb_model_D.best_ntree_limit)

pred_ln_H2 = xgb_model_H.predict(FP_forecast, ntree_limit=xgb_model_H.best_ntree_limit)

pred_ln_G2 = xgb_model_G.predict(FP_forecast, ntree_limit=xgb_model_G.best_ntree_limit)

pred_ln_GS2 = xgb_model_GS.predict(FP_forecast, ntree_limit=xgb_model_GS.best_ntree_limit)

pred_D2 = np.exp(pred_ln_D2)

pred_H2 = np.exp(pred_ln_H2)

pred_G2 = np.exp(pred_ln_G2)

pred_GS2 = np.exp(pred_ln_GS2)

In [ ]:
# Output results for forest stand parameters
output_D2 = np.multiply(ratio_D, pred_D2)
output_H2 = np.multiply(ratio_H, pred_H2)
output_G2 = np.multiply(array_P[:,0], pred_G2)
output_GS2 = np.multiply(array_P[:,0], pred_GS2)

In [ ]:
# Est. biomass use my models

# Prepeare data
biomass_forecast_X = np.c_[array_norm_X_Sp_Or, output_H2, output_D2, output_G2]

In [ ]:
# Load XGBooost models
xgb_model_GS = xgb.XGBRegressor()
xgb_model_allStem = xgb.XGBRegressor()
xgb_model_stemBark = xgb.XGBRegressor()
xgb_model_br = xgb.XGBRegressor()
xgb_model_f = xgb.XGBRegressor()
xgb_model_GS.load_model('./models/biomass_model/xgb_model_LN_GS.json')
xgb_model_allStem.load_model('./models/biomass_model/xgb_model_LN_M_all_stem.json')
xgb_model_stemBark.load_model('./models/biomass_model/xgb_model_LN_M_branch.json')
xgb_model_br.load_model('./models/biomass_model/xgb_model_LN_M_branch.json')
xgb_model_f.load_model('./models/biomass_model/xgb_model_LN_M_stem_bark.json')

#Predict parameters
pred_my_ln_GS = xgb_model_GS.predict(biomass_forecast_X, ntree_limit=xgb_model_GS.best_ntree_limit)
pred_my_ln_allStem = xgb_model_allStem.predict(biomass_forecast_X, ntree_limit=xgb_model_allStem.best_ntree_limit)
pred_my_ln_stemBark = xgb_model_stemBark.predict(biomass_forecast_X, ntree_limit=xgb_model_stemBark.best_ntree_limit)
pred_my_ln_br = xgb_model_br.predict(biomass_forecast_X, ntree_limit=xgb_model_br.best_ntree_limit)
pred_my_ln_f = xgb_model_f.predict(biomass_forecast_X, ntree_limit=xgb_model_f.best_ntree_limit)

In [ ]:
# Convert predicted parameters to real values
pred_my_GS = np.exp(pred_my_ln_GS)
pred_my_allStem = np.exp(pred_my_ln_allStem)
pred_my_stemBark = np.exp(pred_my_ln_stemBark)
pred_my_br = np.exp(pred_my_ln_br)
pred_my_f = np.exp(pred_my_ln_f)

In [ ]:
# Calculating the sum of above-ground biomass using the XGBoost algorithm
pred_my_AG_biomass = pred_my_allStem + pred_my_br + pred_my_f

In [ ]:
# Reshape our predict maps for forest parameters
final_D2 = output_D2.reshape(X_A[:, :, 0].shape)
final_H2 = output_H2.reshape(X_A[:, :, 0].shape)
final_G2 = output_G2.reshape(X_A[:, :, 0].shape)
final_GS2 = output_GS2.reshape(X_A[:, :, 0].shape)

final_my_GS = pred_my_GS.reshape(X_A[:, :, 0].shape)
final_my_allStem = pred_my_allStem.reshape(X_A[:, :, 0].shape)
final_my_stemBark = pred_my_stemBark.reshape(X_A[:, :, 0].shape)
final_my_br = pred_my_br.reshape(X_A[:, :, 0].shape)
final_my_f = pred_my_f.reshape(X_A[:, :, 0].shape)
final_my_AG_biomass = pred_my_AG_biomass.reshape(X_A[:, :, 0].shape)

# *Preporation RN data*

In [ ]:
# RN decay
# calculation 
decay_time = year-2020
decay_Sr = 1*2**(-decay_time/28.91)
decay_Cs = 1*2**(-decay_time/30.08)

In [ ]:
# Convert into array RN bands
X_RN = img[:, :, [0, 1, 2]]

# Take our full image, ignore the Fmask band, and reshape into long 2d array (nrow * ncol, nband) for classification
new_shape_X_RN = (X_RN.shape[0] * X_RN.shape[1], X_RN.shape[2])

array_X_RN = X_RN.reshape(new_shape_X_RN)
print('Reshaped from {o} to {n}'.format(o=X_RN.shape,
                                        n=array_X_RN.shape))

In [ ]:
# Calculation of RN concentration and AEDR as of "year"
output_Sr = array_X_RN[:,0] * decay_Sr
output_Cs = array_X_RN[:,1] * decay_Cs
output_AEDR = ((array_X_RN[:,2]-0.1) * decay_Cs)+0.1

In [ ]:
# Calculation of RN concentration in trunk wood as of "year"

X_Sp = img[:, :, [3]] # select species band
new_shape_X_Sp = (X_Sp.shape[0] * X_Sp.shape[1], X_Sp.shape[2])
array_X_Sp= X_Sp.reshape(new_shape_X_Sp)

X_Fsoil = img[:, :, [9]] # select soil fertility band
new_shape_X_Fsoil = (X_Fsoil.shape[0] * X_Fsoil.shape[1], X_Fsoil.shape[2])
array_X_Fsoil= X_Fsoil.reshape(new_shape_X_Fsoil)
              
# Created array for predict of RN activity concentration of stem wood
Wood_AC_Sr_forecast_X = np.c_[array_X_Sp, output_D2, array_X_Fsoil, output_AEDR, output_Sr]
Wood_AC_Cs_forecast_X = np.c_[array_X_Sp, output_D2, output_H2, output_G2, A_forecast, array_X_Fsoil, output_AEDR]

# Load XGBooost models
xgb_model_ln_Sr_wood = xgb.XGBRegressor()
xgb_model_ln_Cs_wood = xgb.XGBRegressor()

xgb_model_ln_Sr_wood.load_model('./models/RN_models/xgb_model_LN_Sr_without_geo(D).txt')
xgb_model_ln_Cs_wood.load_model('./models/RN_models/xgb_model_LN_Cs_without_geo.txt')

# Predict parameters
pred_ln_Sr_wood = xgb_model_ln_Sr_wood.predict(Wood_AC_Sr_forecast_X, ntree_limit=xgb_model_ln_Sr_wood.best_ntree_limit)
pred_ln_Cs_wood = xgb_model_ln_Cs_wood.predict(Wood_AC_Cs_forecast_X, ntree_limit=xgb_model_ln_Cs_wood.best_ntree_limit)

In [ ]:
# Preprocessing RN and AEDR data before transfer into raster

# Convert predicted parameters to real values
pred_Sr_wood = np.exp(pred_ln_Sr_wood)
pred_Cs_wood = np.exp(pred_ln_Cs_wood)

# Reshape our predict maps for RN contaminated of stem wood
final_Sr_wood = pred_Sr_wood.reshape(X_A[:, :, 0].shape)
final_Cs_wood = pred_Cs_wood.reshape(X_A[:, :, 0].shape)

In [ ]:
# Assessment of radiation doses to tree trunk wood

# load "math" and "stat" libraries
from scipy.stats import norm
import math

# preprocessing
massTree=pred_my_allStem*1000/(output_G2/((math.pi*(output_D2/100)**2)/4))
Sr_in_doses = (0.000594*massTree**0.01435)*pred_Sr_wood
Cs_in_doses = (0.0001764**massTree**0.0934)*pred_Cs_wood
In_doses = Sr_in_doses + Cs_in_doses
Ex_doses = output_AEDR
Total_dose = In_doses + Ex_doses
Part_in_dose = In_doses / Total_dose * 100

final_Sr_in_doses = Sr_in_doses.reshape(X_A[:, :, 0].shape)
final_Cs_in_doses = Cs_in_doses.reshape(X_A[:, :, 0].shape)
final_In_doses = In_doses.reshape(X_A[:, :, 0].shape)
final_Ex_doses = Ex_doses.reshape(X_A[:, :, 0].shape)
final_Total_dose = Total_dose.reshape(X_A[:, :, 0].shape)
final_Part_in_dose = Part_in_dose.reshape(X_A[:, :, 0].shape)

In [ ]:
# Prediction of the trees/woods portion that is not exceeding the permissible levels of wood contamination
# for Sr-90
ratio_Sr_wood = pred_Sr_wood/PL_Sr
Z_scope_Sr_wood = np.log(ratio_Sr_wood)/np.log(1.6)
Part_wood_to_PL_Sr = norm(loc = 0 , scale = 1).cdf(Z_scope_Sr_wood)

# for Cs-137
ratio_Cs_wood = pred_Cs_wood/PL_Cs
Z_scope_Cs_wood = np.log(ratio_Cs_wood)/np.log(1.6)
Part_wood_to_PL_Cs = norm(loc = 0 , scale = 1).cdf(Z_scope_Cs_wood)

# final output values
final_Part_wood_to_PL_Sr = Part_wood_to_PL_Sr.reshape(X_A[:, :, 0].shape)
final_Part_wood_to_PL_Cs = Part_wood_to_PL_Cs.reshape(X_A[:, :, 0].shape)

In [ ]:
# Clean up RAM
clean = []

X_SI = clean
new_shape_X_SI = clean
array_X_SI = clean 

X_Sp_Or = clean
new_shape_norm_X_Sp_Or = clean
array_norm_X_Sp_Or = clean
new_shape_norm_X_A = clean

pred_ln_D1 = clean
pred_ln_H1 = clean
pred_ln_G1 = clean
pred_ln_GS1 = clean

D = clean
new_shape_D = clean
array_D = clean
ratio_D = clean
H = clean
new_shape_H = clean
array_H = clean
ratio_H = clean
P = clean
new_shape_P = clean
array_P = clean
G = clean
GS = clean

FP_forecast = clean

pred_ln_D2 = clean
pred_ln_H2 = clean
pred_ln_G2 = clean
pred_ln_GS2 = clean
pred_D2 = clean
pred_H2 = clean
pred_G2 = clean
pred_GS2 = clean

biomass_forecast_X = clean

pred_my_ln_GS = clean
pred_my_ln_allStem = clean
pred_my_ln_stemBark = clean
pred_my_ln_br = clean
pred_my_ln_f = clean

X_RN = clean
new_shape_X_RN = clean
array_X_RN = clean
output_Sr = clean
output_Cs = clean
output_AEDR = clean

X_Sp = clean
new_shape_X_Sp = clean
X_Fsoil = clean
new_shape_X_Fsoil = clean
array_X_Fsoil = clean


Wood_AC_Sr_forecast_X = clean
Wood_AC_Cs_forecast_X = clean

massTree = clean
Sr_in_doses = clean
Cs_in_doses = clean
In_doses = clean
Ex_doses = clean
Total_dose = clean
Part_in_dose = clean

ratio_Sr_wood = clean
Z_scope_Sr_wood = clean
Part_wood_to_PL_Sr = clean
ratio_Cs_wood = clean
Z_scope_Cs_wood = clean
Part_wood_to_PL_Cs = clean

# Est. of RN stock in AG biomass

In [ ]:
# Reclassfication species_ID
neg_invers_array_X_Sp = array_X_Sp * -1

In [ ]:
# Ratio between RN activity concentration in biomass component and trunk wood ([trunk bark, branch, foliage])
ratio_Cs_Pine = [3, 3, 5]
ratio_Sr_Pine = [4, 3.5, 3]
ratio_Cs_Spruce = [3, 3, 5]
ratio_Sr_Spruce = [4, 3.5, 3]
ratio_Cs_Birch = [2, 2, 3]
ratio_Sr_Birch = [6, 5, 7]
ratio_Cs_Alder = [1.8, 2, 3]
ratio_Sr_Alder = [2, 2, 2.5]
ratio_Cs_Aspen = [1.8, 2, 3]
ratio_Sr_Aspen = [2, 2, 2.5]
ratio_Cs_Oak = [4, 1.5, 6.5]
ratio_Sr_Oak = [11, 4, 3.5]

# The inverse of species ID 
species_name = [0, -1, -2, -3, -4, -5]

In [ ]:
# Func. for full reclassification data
def est_ratio (ratio_data, raster, species):
    for i in range(len(ratio_data)):
        global a
        a=ratio_data
        globals()[f'comp_{i}'] = np.where(raster == species, ratio_data[i], 0)      

In [ ]:
# Calc. ratio for RN in different components
# for Cs-137
est_ratio(ratio_Cs_Pine, neg_invers_array_X_Sp, 0)
ratio_Pine_comp_0 = comp_0
ratio_Pine_comp_1 = comp_1
ratio_Pine_comp_2 = comp_2

est_ratio(ratio_Cs_Spruce, neg_invers_array_X_Sp, -1)
ratio_Spruce_comp_0 = comp_0
ratio_Spruce_comp_1 = comp_1
ratio_Spruce_comp_2 = comp_2

est_ratio(ratio_Cs_Birch, neg_invers_array_X_Sp, -2)
ratio_Birch_comp_0 = comp_0
ratio_Birch_comp_1 = comp_1
ratio_Birch_comp_2 = comp_2

est_ratio(ratio_Cs_Alder, neg_invers_array_X_Sp, -3)
ratio_Alder_comp_0 = comp_0
ratio_Alder_comp_1 = comp_1
ratio_Alder_comp_2 = comp_2

est_ratio(ratio_Cs_Aspen, neg_invers_array_X_Sp, -4)
ratio_Aspen_comp_0  = comp_0
ratio_Aspen_comp_1 = comp_1
ratio_Aspen_comp_2 = comp_2

est_ratio(ratio_Cs_Oak, neg_invers_array_X_Sp, -5)
ratio_Oak_comp_0 = comp_0
ratio_Oak_comp_1 = comp_1
ratio_Oak_comp_2 = comp_2

ratio_Cs_comp_0 = ratio_Pine_comp_0 + ratio_Spruce_comp_0 + ratio_Birch_comp_0 + ratio_Alder_comp_0 + ratio_Aspen_comp_0 + ratio_Oak_comp_0
ratio_Cs_comp_1 = ratio_Pine_comp_1 + ratio_Spruce_comp_1 + ratio_Birch_comp_1 + ratio_Alder_comp_1 + ratio_Aspen_comp_1 + ratio_Oak_comp_1
ratio_Cs_comp_2 = ratio_Pine_comp_2 + ratio_Spruce_comp_2 + ratio_Birch_comp_2 + ratio_Alder_comp_2 + ratio_Aspen_comp_2 + ratio_Oak_comp_2

# for Sr-90
est_ratio(ratio_Sr_Pine, neg_invers_array_X_Sp, 0)
ratio_Pine_comp_0 = comp_0
ratio_Pine_comp_1 = comp_1
ratio_Pine_comp_2 = comp_2

est_ratio(ratio_Sr_Spruce, neg_invers_array_X_Sp, -1)
ratio_Spruce_comp_0 = comp_0
ratio_Spruce_comp_1 = comp_1
ratio_Spruce_comp_2 = comp_2

est_ratio(ratio_Sr_Birch, neg_invers_array_X_Sp, -2)
ratio_Birch_comp_0 = comp_0
ratio_Birch_comp_1 = comp_1
ratio_Birch_comp_2 = comp_2

est_ratio(ratio_Sr_Alder, neg_invers_array_X_Sp, -3)
ratio_Alder_comp_0 = comp_0
ratio_Alder_comp_1 = comp_1
ratio_Alder_comp_2 = comp_2

est_ratio(ratio_Sr_Aspen, neg_invers_array_X_Sp, -4)
ratio_Aspen_comp_1  = comp_0
ratio_Aspen_comp_1 = comp_1
ratio_Aspen_comp_2 = comp_2

est_ratio(ratio_Sr_Oak, neg_invers_array_X_Sp, -5)
ratio_Oak_comp_0 = comp_0
ratio_Oak_comp_1 = comp_1
ratio_Oak_comp_2 = comp_2

ratio_Sr_comp_0 = ratio_Pine_comp_0 + ratio_Spruce_comp_0 + ratio_Birch_comp_0 + ratio_Alder_comp_0 + ratio_Aspen_comp_0 + ratio_Oak_comp_0
ratio_Sr_comp_1 = ratio_Pine_comp_1 + ratio_Spruce_comp_1 + ratio_Birch_comp_1 + ratio_Alder_comp_1 + ratio_Aspen_comp_1 + ratio_Oak_comp_1
ratio_Sr_comp_2 = ratio_Pine_comp_2 + ratio_Spruce_comp_2 + ratio_Birch_comp_2 + ratio_Alder_comp_2 + ratio_Aspen_comp_2 + ratio_Oak_comp_2

# Clean used dataset
comp_0 = clean
comp_1 = clean
comp_2 = clean
ratio_Pine_comp_0 = clean
ratio_Pine_comp_1 = clean
ratio_Pine_comp_2 = clean
ratio_Spruce_comp_0 = clean
ratio_Spruce_comp_1 = clean
ratio_Spruce_comp_2 = clean
ratio_Birch_comp_0 = clean
ratio_Birch_comp_1 = clean
ratio_Birch_comp_2 = clean
ratio_Alder_comp_0 = clean
ratio_Alder_comp_1 = clean
ratio_Alder_comp_2 = clean
ratio_Aspen_comp_0  = clean
ratio_Aspen_comp_1 = clean
ratio_Aspen_comp_2 = clean
ratio_Oak_comp_0 = clean
ratio_Oak_comp_1 = clean
ratio_Oak_comp_2 = clean

In [ ]:
# Transpose of NumPy array

ratio_Sr_comp_0 = np.transpose(ratio_Sr_comp_0)
ratio_Sr_comp_1 = np.transpose(ratio_Sr_comp_1)
ratio_Sr_comp_2 = np.transpose(ratio_Sr_comp_2)
ratio_Cs_comp_0 = np.transpose(ratio_Cs_comp_0)
ratio_Cs_comp_1 = np.transpose(ratio_Cs_comp_1)
ratio_Cs_comp_2 = np.transpose(ratio_Cs_comp_2)

In [ ]:
# Calculate RN stock in tree

# Est. activity consentration RN
pred_Sr_bark = pred_Sr_wood * ratio_Sr_comp_0
pred_Cs_bark = pred_Cs_wood * ratio_Cs_comp_0

pred_Sr_br = pred_Sr_wood * ratio_Sr_comp_1
pred_Cs_br = pred_Cs_wood * ratio_Cs_comp_1

pred_Sr_f = pred_Sr_wood * ratio_Sr_comp_2
pred_Cs_f = pred_Cs_wood * ratio_Cs_comp_2

# Clean used dataset
ratio_Sr_comp_0 = clean
ratio_Sr_comp_1 = clean
ratio_Sr_comp_2 = clean
ratio_Cs_comp_0 = clean
ratio_Cs_comp_1 = clean
ratio_Cs_comp_2 = clean

#Est. RN stock in different above-ground parts of trees (kBq/sq.m)
stock_Sr_stem_wood = pred_Sr_wood*(pred_my_allStem - pred_my_stemBark)/10000
stock_Sr_stem_bark = pred_Sr_bark*pred_my_stemBark/10000
stock_Sr_br = pred_Sr_br*pred_my_br/10000
stock_Sr_f = pred_Sr_f*pred_my_f/10000
stock_Sr_AG = stock_Sr_stem_wood + stock_Sr_stem_bark + stock_Sr_br + stock_Sr_f
stock_Cs_stem_wood = pred_Cs_wood*(pred_my_allStem - pred_my_stemBark)/10000
stock_Cs_stem_bark = pred_Cs_bark*pred_my_stemBark/10000
stock_Cs_br = pred_Cs_br*pred_my_br/10000
stock_Cs_f = pred_Cs_f*pred_my_f/10000
stock_Cs_AG = stock_Cs_stem_wood + stock_Cs_stem_bark + stock_Cs_br + stock_Cs_f

final_stock_Sr_stem_wood = stock_Sr_stem_wood.reshape(X_A[:, :, 0].shape)
final_stock_Sr_stem_bark = stock_Sr_stem_bark.reshape(X_A[:, :, 0].shape)
final_stock_Sr_br = stock_Sr_br.reshape(X_A[:, :, 0].shape)
final_stock_Sr_f = stock_Sr_f.reshape(X_A[:, :, 0].shape)
final_stock_Sr_AG = stock_Sr_AG.reshape(X_A[:, :, 0].shape)
final_stock_Cs_stem_wood = stock_Cs_stem_wood.reshape(X_A[:, :, 0].shape)
final_stock_Cs_stem_bark = stock_Cs_stem_bark.reshape(X_A[:, :, 0].shape)
final_stock_Cs_br = stock_Cs_br.reshape(X_A[:, :, 0].shape)
final_stock_Cs_f = stock_Cs_f.reshape(X_A[:, :, 0].shape)
final_stock_Cs_AG = stock_Cs_AG.reshape(X_A[:, :, 0].shape)

# Clean used dataset
pred_Sr_bark = clean
pred_Cs_bark = clean
pred_Sr_br = clean
pred_Cs_br = clean
pred_Sr_f = clean
pred_Cs_f = clean
stock_Sr_stem_wood = clean
stock_Sr_stem_bark = clean
stock_Sr_br = clean
stock_Sr_f = clean
stock_Sr_AG = clean
stock_Cs_stem_wood = clean
stock_Cs_stem_bark = clean
stock_Cs_br = clean
stock_Cs_f = clean
stock_Cs_AG = clean

# ***Estimation Pine forest litter parameters***

In [ ]:
# Est. of forest floor litter in Pine stands use XGBoost model

# Prepeare data
litter_forecast_X = np.c_[output_H2, output_D2, output_G2]

In [ ]:
# Load XGBooost models
xgb_model_Litter = xgb.XGBRegressor()
xgb_model_Litter.load_model('./models/litter_model/XGB_model_litter_biomass.json')

#Predict forest litter biomass
pred_Litter_biomass = xgb_model_Litter.predict(litter_forecast_X, ntree_limit=xgb_model_Litter.best_ntree_limit)

In [ ]:
# Estimation of RN inventory  (kBq/sq.m)

litter_Sr_L = pred_Litter_biomass * 0.00258 * pred_Sr_wood
litter_Sr_F = pred_Litter_biomass * 0.00303 * pred_Sr_wood
litter_Sr_H = pred_Litter_biomass * 0.00602 * pred_Sr_wood

litter_Cs_L = pred_Litter_biomass * 0.00235 * pred_Cs_wood
litter_Cs_F = pred_Litter_biomass * 0.00311 * pred_Cs_wood
litter_Cs_H = pred_Litter_biomass * 0.00727 * pred_Cs_wood

Fuel_litter_Sr = litter_Sr_L + litter_Sr_F
Fuel_litter_Cs = litter_Cs_L + litter_Cs_F

Total_litter_Sr = litter_Sr_L + litter_Sr_F + litter_Sr_H
Total_litter_Cs = litter_Cs_L + litter_Cs_F + litter_Cs_H

# Clear data set
litter_forecast_X = clean

litter_Sr_L = clean
litter_Sr_F = clean
litter_Sr_H = clean

litter_Cs_L = clean
litter_Cs_F = clean
litter_Cs_H = clean

In [ ]:
# Reshape our predict maps for forest litter
final_Litter_biomass = pred_Litter_biomass.reshape(X_A[:, :, 0].shape)
final_Fuel_litter_Sr = Fuel_litter_Sr.reshape(X_A[:, :, 0].shape)
final_Fuel_litter_Cs = Fuel_litter_Cs.reshape(X_A[:, :, 0].shape)
final_Total_litter_Sr = Total_litter_Sr.reshape(X_A[:, :, 0].shape)
final_Total_litter_Cs = Total_litter_Cs.reshape(X_A[:, :, 0].shape)

# ***Save all rasters***

In [ ]:
# Function for save output rasters with results (for maps)
def save_map (file_location, layer):
    with rasterio.open(
        file_location,
        'w',
        driver='GTiff',
        height=layer.shape[0],
        width=layer.shape[1],
        count=1,
        dtype=np.float32,
        transform=af_transf
    ) as dest_file:
        dest_file.write(layer, 1)
    dest_file.close()    

In [ ]:
# Save forest parameters to map / single-band raster
# forest stand indicators
save_map (f'./output_rasters/01_DBH_cm_{year}.tif', final_D2)
save_map (f'./output_rasters/02_H_m_{year}.tif', final_H2)
save_map (f'./output_rasters/03_G_or_BA_sq_m_ha{year}.tif', final_G2)
save_map (f'./output_rasters/04_classical_GS_cub_m_ha_{year}.tif', final_GS2)
save_map (f'./output_rasters/05_my_GS_cub_m_ha_{year}.tif', final_my_GS)
# above-ground biomass
save_map (f'./output_rasters/06_all_stem_biomass_t_ha_{year}.tif', final_my_allStem)
save_map (f'./output_rasters/07_stem_bark_biomass_t_ha_{year}.tif', final_my_stemBark)
save_map (f'./output_rasters/08_branch_biomass_t_ha_{year}.tif', final_my_br)
save_map (f'./output_rasters/09_foliage_biomass_t_ha_{year}.tif', final_my_f)
save_map (f'./output_rasters/10_AG_biomass_t_ha_{year}.tif', final_my_AG_biomass)
# RN activity concentration in woods
save_map (f'./output_rasters/11_Sr_content_in_wood_Bq_kg_{year}.tif', final_Sr_wood)
save_map (f'./output_rasters/12_Cs_content_in_wood_Bq_kg_{year}.tif', final_Cs_wood)
# Propobility is not exceeding the permissible levels for woood
save_map (f'./output_rasters/13_P_Sr_content>PL_(0-1)_{year}.tif', final_Part_wood_to_PL_Sr)
save_map (f'./output_rasters/14_P_Cs_content>PL_(0-1)_{year}.tif', final_Part_wood_to_PL_Cs)
# Doses
save_map (f'./output_rasters/15_Sr_in_dose_µGy_h_{year}.tif', final_Sr_in_doses)
save_map (f'./output_rasters/16_Cs_in_dose_µGy_h_{year}.tif', final_Cs_in_doses)
save_map (f'./output_rasters/17_Internal dose_µGy_h_{year}.tif', final_In_doses)
save_map (f'./output_rasters/18_Ex_dose_µGy_h_{year}.tif', final_Ex_doses)
save_map (f'./output_rasters/19_Sr_Total_dose_µGy_h_{year}.tif', final_Total_dose)
save_map (f'./output_rasters/20_Cs_Part_internal_dose_(%)_{year}.tif', final_Part_in_dose)
# RN inventory in above-ground biomass components
save_map (f'./output_rasters/21_D_activity_Sr_stem_wood_kBq_sq_m_{year}.tif', final_stock_Sr_stem_wood)
save_map (f'./output_rasters/22_D_activity_Sr_bark_wood_kBq_sq_m_{year}.tif', final_stock_Sr_stem_bark)
save_map (f'./output_rasters/23_D_activity_Sr_branch_kBq_sq_m_{year}.tif', final_stock_Sr_br)
save_map (f'./output_rasters/24_D_activity_Sr_foliage_kBq_sq_m_{year}.tif', final_stock_Sr_f)
save_map (f'./output_rasters/25_D_activity_Sr_AG_kBq_sq_m_{year}.tif', final_stock_Sr_AG)
save_map (f'./output_rasters/26_D_activity_Cs_stem_wood_kBq_sq_m_{year}.tif', final_stock_Cs_stem_wood)
save_map (f'./output_rasters/27_D_activity_Cs_stem_bark_kBq_sq_m_{year}.tif', final_stock_Cs_stem_bark)
save_map (f'./output_rasters/29_D_activity_Cs_branch_kBq_sq_m_{year}.tif', final_stock_Cs_br)
save_map (f'./output_rasters/30_D_activity_Cs_foliage_kBq_sq_m{year}.tif', final_stock_Cs_f)
save_map (f'./output_rasters/31_D_activity_Cs_AG_kBq_sq_m_{year}.tif', final_stock_Cs_AG)
# Forest litter
save_map (f'./output_rasters/32_Litter_biomass_kg_sq_m_(only_Pine)_{year}.tif', final_Litter_biomass)
save_map (f'./output_rasters/33_Fuel_litter_activity_Sr_kBq_sq_m_(only_Pine)_{year}.tif', final_Fuel_litter_Sr)
save_map (f'./output_rasters/34_Fuel_litter_activity_Cs_kBq_sq_m_(only_Pine)_{year}.tif', final_Fuel_litter_Cs)
save_map (f'./output_rasters/35_Total_litter_activity_Sr_kBq_sq_m_(only_Pine)_{year}.tif', final_Total_litter_Sr)
save_map (f'./output_rasters/36_Total_litter_activity_Cs_kBq_sq_m_(only_Pine)_{year}.tif', final_Total_litter_Cs)

print(' *** All rasters saved *** ')